In [184]:
#!g1.1
train = pd.read_csv('/home/jupyter/mnt/s3/bank-data/data/train.csv')
train

,Unnamed: 0,sentence,1category,2category,sentiment
0,4754,При этом всегда получал качественные услуги.,Communication,NaN,+
1,4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,NaN,−
2,3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,NaN,−
3,11640,Отвратительное отношение к клиентам.,Communication,NaN,−
4,5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,NaN,+
...,...,...,...,...,...
19356,8004,Никогда и ни в коем случае не открывайте счет ...,Communication,NaN,−
19357,18182,ТИ откровенно забили на качество и развивают с...,Quality,NaN,−
19358,744,"Я считаю, это прорыв и лидерство финансовых ус...",?,NaN,+
19359,6220,"Писал мужчина очень доходчиво, не финансовым я...",Communication,NaN,+


In [185]:
#!g1.1
emb_sbert = pd.read_csv('/home/jupyter/mnt/s3/bank-data/features/embeddings_sbert.csv')
emb_sbert

,sbert_0,sbert_1,sbert_2,sbert_3,sbert_4,sbert_5,sbert_6,sbert_7,sbert_8,sbert_9,sbert_10,sbert_11,sbert_12,sbert_13,sbert_14,sbert_15,sbert_16,sbert_17,sbert_18,sbert_19,sbert_20,sbert_21,sbert_22,sbert_23,sbert_24,sbert_25,sbert_26,sbert_27,sbert_28,sbert_29,sbert_30,sbert_31,sbert_32,sbert_33,sbert_34,sbert_35,sbert_36,sbert_37,sbert_38,sbert_39,...,sbert_728,sbert_729,sbert_730,sbert_731,sbert_732,sbert_733,sbert_734,sbert_735,sbert_736,sbert_737,sbert_738,sbert_739,sbert_740,sbert_741,sbert_742,sbert_743,sbert_744,sbert_745,sbert_746,sbert_747,sbert_748,sbert_749,sbert_750,sbert_751,sbert_752,sbert_753,sbert_754,sbert_755,sbert_756,sbert_757,sbert_758,sbert_759,sbert_760,sbert_761,sbert_762,sbert_763,sbert_764,sbert_765,sbert_766,sbert_767
0,-0.260432,-0.713181,0.003374,-0.555725,-0.040731,-0.649877,0.681974,-0.492740,0.182050,-0.232997,-0.812267,0.105695,-0.658094,0.458973,0.376190,-0.546793,0.880897,-1.016082,-0.273517,0.086956,0.107551,-0.566165,-0.579776,0.868737,-0.350201,0.175540,0.197782,-0.576191,0.394082,-0.528489,0.534617,0.874266,-0.994191,1.475890,-1.130669,-0.193522,-0.112922,1.634456,-0.306568,-0.049407,...,0.388499,0.140692,-1.000657,-1.748767,0.102634,-0.213602,-0.171211,-0.433328,-0.001321,-0.855906,0.500770,0.046936,-0.368081,-0.608188,-0.487786,0.271716,-0.486945,-0.293933,0.557210,1.024988,0.285357,-1.330798,-0.071485,0.678186,-0.363151,0.428721,0.177887,0.320728,-0.669128,-0.331660,-0.564299,0.232505,-0.150469,-0.380158,-1.243876,-0.661827,0.497174,0.294333,-0.757891,0.929163
1,-0.267600,-1.328871,-0.050176,-0.032681,-1.467123,-0.453183,0.874981,-0.442528,0.511680,-0.914371,-0.789241,0.226640,-0.135146,-0.524694,0.640999,-0.992747,0.694822,0.373403,-0.745101,-0.179952,1.469381,-0.136895,-0.476601,-0.096928,-0.065097,-0.116225,1.149384,-0.059491,0.803913,0.262802,-0.057495,0.341050,0.125225,-0.036525,-0.833674,0.464959,-0.687025,1.177489,-0.596040,-0.265125,...,-0.024948,0.425574,-1.413946,-0.872738,0.081779,0.340101,0.343509,-0.586604,-0.104713,-0.274364,0.364234,-0.167066,0.285902,-0.903708,0.106425,0.544268,-0.215614,-0.021912,0.408168,-0.048864,-0.199491,-0.280570,0.077384,0.396974,-1.319457,-0.282201,0.182202,0.160199,0.567249,0.793851,0.791312,0.710322,-0.019681,-0.284321,0.235238,0.326818,-0.435732,-0.098490,-0.766283,0.991424
2,-0.638397,-1.266012,0.243861,-1.019216,-0.645469,-0.181478,0.467928,0.406027,-0.059911,-0.367818,0.731275,0.345900,-1.103809,-0.183755,0.183583,-0.314409,0.761252,0.880600,-0.586889,-0.224529,1.081648,-0.425653,-0.274231,1.018959,0.529894,0.212872,1.261395,0.293828,0.920499,0.788032,-0.235910,0.313926,-1.023695,0.015049,-0.984671,-0.460414,-0.605235,1.829781,0.042270,-0.407832,...,-0.249223,-0.010058,-1.423890,-1.368226,0.820630,1.015616,-0.345912,-0.846165,0.253047,-0.593225,-0.252492,0.276455,0.325634,-0.175439,-0.416424,0.230547,-0.124597,-0.137502,1.653867,0.133794,-0.451375,0.070482,0.404447,0.356175,-0.992515,-0.634342,-0.751792,-0.065858,-0.662084,-0.676172,0.375245,0.540558,0.211533,0.653911,0.285135,-0.246547,0.034842,0.374983,-0.243360,1.456757
3,-0.187049,-1.172277,-0.252848,-0.020091,-1.145047,-0.293424,0.796849,-0.893079,0.184771,-0.879934,-0.604819,0.378867,-0.275533,-0.516072,0.163134,-0.513828,0.666621,-0.176098,-0.085796,-1.278306,0.593058,-0.164535,-0.908730,0.594817,0.423382,-0.573923,0.372415,0.206064,0.970025,0.666710,0.069733,-0.145418,-0.951699,0.404901,-0.695958,0.250444,0.353053,1.656136,0.011415,-0.654960,...,0.510405,0.655207,-1.747272,-1.930507,-0.030805,0.264029,0.272147,-0.567929,-0.042941,-0.333802,0.600960,0.344754,-1.493081,-0.993676,-0.264971,0.602269,0.329934,0.289166,0.312787,0.203136,-0.187067,-1.304461,1.102900,1.109003,-0.731355,-0.026351,0.244357,0.156108,-0.524889,-0.264964,-0.488474,0.525499,0.113292,-0.562937,-0.564802,-0.684930,0.296284,0.385324,-0.702113,1.304390
4,-0.622859,-0.543196,0.239860,-0.251940,-0.344115,-0.174159,0.728634,-0.675047,1.129310,-0.788626,-0.637421,0.142306,1.535083,-0.247185,-0.406887,-0.307516,1.470772,

In [186]:
#!g1.1
dist_sent_final = pd.read_csv('/home/jupyter/mnt/s3/bank-data/features/distances_sentiments_FINAL.csv')
dist_sent_final

,Unnamed: 0,sentence,dist_pos_0,dist_pos_1,dist_pos_2,dist_pos_3,dist_pos_4,dist_pos_5,dist_pos_6,dist_pos_7,dist_pos_8,dist_pos_9,dist_pos_10,dist_pos_11,dist_pos_12,dist_pos_13,dist_pos_14,dist_pos_15,dist_pos_16,dist_pos_17,dist_pos_18,dist_pos_19,dist_pos_20,dist_pos_21,dist_pos_22,dist_pos_23,dist_pos_24,dist_pos_25,dist_pos_26,dist_pos_27,dist_pos_28,dist_pos_29,dist_pos_30,dist_pos_31,dist_pos_32,dist_pos_33,dist_pos_34,dist_pos_35,dist_neg_0,dist_neg_1,...,dist_neg_4,dist_neg_5,dist_neg_6,dist_neg_7,dist_neg_8,dist_neg_9,dist_neg_10,dist_neg_11,dist_neg_12,dist_neg_13,dist_neg_14,dist_neg_15,dist_neg_16,dist_neg_17,dist_net_0,dist_net_1,dist_net_2,dist_net_3,dist_net_4,dist_net_5,dist_net_6,dist_net_7,dist_net_8,dist_net_9,dist_net_10,dist_net_11,dist_net_12,dist_net_13,dist_net_14,dist_net_15,dist_net_16,dist_net_17,dist_net_18,dist_net_19,dist_net_20,dist_net_21,dist_net_22,dist_net_23,dist_net_24,dist_net_25
0,0,При этом всегда получал качественные услуги.,0.836873,0.604573,0.698533,0.590854,0.653068,0.805598,0.752680,0.785493,0.997203,0.704370,0.611304,0.563475,0.761309,0.561013,0.801724,0.664638,0.808556,0.607513,0.698246,0.636207,0.706370,0.697671,0.438187,0.677315,0.739961,0.756131,0.886441,0.786875,0.711190,0.664307,0.633743,0.584369,0.642824,0.689954,0.791031,0.511739,0.681406,0.766079,...,0.632271,0.826995,0.819750,0.736176,0.848231,0.660029,0.456691,0.759870,0.474292,0.780699,0.634758,0.423451,0.797976,0.680918,0.440418,0.794999,0.570097,0.745708,0.456691,0.573373,0.722134,0.675660,0.604929,0.681406,0.924446,0.698424,0.551746,0.794674,0.625063,0.660604,0.760979,0.739961,0.817767,0.814103,0.744761,0.541154,0.584369,0.726199,0.842304,0.952256
1,1,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",0.802712,0.602334,0.846544,0.576117,0.695180,0.815178,0.800020,0.767373,0.968035,0.719439,0.746809,0.699854,0.756856,0.603126,0.854975,0.712624,0.846889,0.711870,0.580997,0.723233,0.737448,0.723575,0.483884,0.783290,0.825369,0.713520,0.904082,0.805403,0.727418,0.694732,0.590932,0.630424,0.821721,0.735185,0.859307,0.715081,0.698506,0.730816,...,0.569874,0.814389,0.959843,0.819716,0.766667,0.605005,0.482961,0.881801,0.474791,0.734759,0.714265,0.417713,0.710720,0.779665,0.491570,0.554447,0.582721,0.664956,0.482961,0.696643,0.777495,0.699734,0.840832,0.698506,0.826977,0.781379,0.550116,0.770550,0.532063,0.712268,0.812895,0.825369,0.872314,0.780906,0.821223,0.525663,0.630424,0.723414,0.855830,0.893772
2,2,"Вот так ""Мой любимый"" банк МКБ меня обманул.",0.957085,0.894284,0.914250,0.759423,0.841217,0.844626,0.673352,0.824179,0.806325,0.882951,0.707285,0.576256,0.813853,0.770653,0.879613,0.890761,0.833160,0.653186,0.742756,0.848832,0.725804,0.699959,0.824627,0.925448,0.785118,0.816599,0.925141,0.986610,0.891091,0.821176,0.728249,0.539725,0.901882,0.707264,0.877733,0.869060,0.820585,0.812604,...,0.839637,0.661302,0.803044,0.706408,0.745217,0.916141,0.659354,0.765459,0.606467,0.725298,0.761001,0.744157,0.669902,0.607836,0.705759,0.711081,0.779243,0.779992,0.659354,0.720244,0.800440,0.910170,0.810957,0.820585,0.824958,0.740775,0.862632,0.895270,0.895211,0.719754,0.965058,0.785118,0.687162,0.782587,0.718057,0.657573,0.539725,0.714284,0.785402,0.864692
3,3,Отвратительное отношение к клиентам.,0.829918,0.688374,0.777411,0.628393,0.821164,0.850740,0.504458,0.811635,1.090454,0.663675,0.713983,0.712763,0.699118,0.777538,0.774825,0.768650,0.738199,0.725874,0.793414,0.552334,0.779744,0.837066,0.718993,0.822136,0.834056,0.807777,1.020236,0.887929,0.817368,0.791998,0.799160,0.659338,0.704996,0.695038,0.919692,0.682002,0.807196,0.808811,...,0.762714,0.629532,0.847164,0.258026,0.751795,0.737491,0.720639,0.000000,0.683376,0.937594,0.841082,0.766761,0.825767,0.782251,0.705851,0.639173,0.763197,0.853429,0.720639,0.794713,0.745125,0.765460,0.831290,0.807196,0.973786,0.891967,0.792868,0.908751,0.833514,0.798442,0.865200,0.834056,0.818867,0.727231,0.871679,0.828155,0.659338,0.830752,0.907560,0.967083
4,4,"Всегда в любое время дня и ночи помогут

In [187]:
#!g1.1
dist_cat_final = pd.read_csv('/home/jupyter/mnt/s3/bank-data/features/distances_category_FINAL.csv')
dist_cat_final

,Unnamed: 0,sentence,dist_comm_0,dist_comm_1,dist_comm_2,dist_comm_3,dist_comm_4,dist_comm_5,dist_comm_6,dist_comm_7,dist_comm_8,dist_comm_9,dist_comm_10,dist_comm_11,dist_comm_12,dist_comm_13,dist_comm_14,dist_comm_15,dist_comm_16,dist_qual_0,dist_qual_1,dist_qual_2,dist_qual_3,dist_qual_4,dist_qual_5,dist_qual_6,dist_qual_7,dist_qual_8,dist_qual_9,dist_qual_10,dist_qual_11,dist_price_0,dist_price_1,dist_price_2,dist_price_3,dist_price_4,dist_price_5,dist_price_6,dist_price_7,dist_price_8,...,dist_safe_9,dist_safe_10,dist_safe_11,dist_safe_12,dist_safe_13,dist_safe_14,dist_safe_15,dist_safe_16,dist_safe_17,dist_safe_18,dist_safe_19,dist_safe_20,dist_safe_21,dist_safe_22,dist_safe_23,dist_safe_24,dist_safe_25,dist_safe_26,dist_safe_27,dist_safe_28,dist_safe_29,dist_safe_30,dist_safe_31,dist_safe_32,dist_safe_33,dist_safe_34,dist_safe_35,dist_safe_36,dist_safe_37,dist_safe_38,dist_safe_39,dist_safe_40,dist_safe_41,dist_safe_42,dist_safe_43,dist_safe_44,dist_safe_45,dist_safe_46,dist_safe_47,dist_safe_48
0,0,При этом всегда получал качественные услуги.,0.680918,0.561013,0.456691,0.607513,0.639530,0.632271,0.741922,0.681406,0.578317,0.863590,0.704370,0.826995,0.660604,0.634758,0.916695,0.760979,0.808556,0.997203,0.767193,0.808670,0.689954,0.756131,0.736176,0.759870,0.799069,0.706370,0.797976,0.536052,0.862672,0.820153,0.604929,0.789692,0.744761,0.712839,0.672078,0.665457,0.933083,0.908264,...,0.734937,0.674532,0.491478,0.874875,0.603384,0.778237,0.703469,0.785196,0.708626,0.741922,0.731976,0.681406,0.532674,0.630616,0.650280,0.812172,0.605427,0.797976,0.683069,0.647447,0.674094,0.675660,0.639530,0.763256,0.747876,0.660604,0.671792,0.817767,0.709259,0.565704,0.515798,0.719343,0.578040,0.760536,0.716407,0.554981,0.757976,0.784912,0.729447,0.711379
1,1,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",0.779665,0.603126,0.482961,0.711870,0.569350,0.569874,0.705125,0.698506,0.826086,0.717897,0.719439,0.814389,0.712268,0.714265,0.858154,0.812895,0.846889,0.968035,0.818615,0.843531,0.735185,0.713520,0.819716,0.881801,0.845672,0.737448,0.710720,0.797201,0.775480,0.983115,0.840832,0.847568,0.821223,0.685803,0.810866,0.611915,0.899726,0.864427,...,0.735952,0.695100,0.523307,0.901972,0.679898,0.820604,0.791640,0.788055,0.777826,0.705125,0.756532,0.698506,0.572157,0.585323,0.745696,0.759397,0.530999,0.710720,0.774404,0.639763,0.637532,0.699734,0.569350,0.871754,0.904471,0.712268,0.695816,0.872314,0.857348,0.702126,0.597942,0.677619,0.760695,0.680339,0.828524,0.000000,0.827509,0.817264,0.804307,0.787932
2,2,"Вот так ""Мой любимый"" банк МКБ меня обманул.",0.607836,0.770653,0.659354,0.653186,0.684481,0.839637,0.000000,0.820585,0.875251,0.851932,0.882951,0.661302,0.719754,0.761001,0.900134,0.965058,0.833160,0.806325,0.899832,0.816332,0.707264,0.816599,0.706408,0.765459,0.838414,0.725804,0.669902,0.810448,0.683507,0.923471,0.810957,0.786894,0.718057,0.831548,0.707838,0.806565,0.891932,0.743829,...,0.802765,0.639942,0.735739,0.668307,0.770944,0.964916,0.722955,0.799753,0.751855,0.000000,0.812005,0.820585,0.754897,0.783974,0.932115,0.359387,0.743419,0.669902,0.826921,0.690277,0.798570,0.910170,0.684481,0.773862,0.897507,0.719754,0.806423,0.687162,0.847687,0.675053,0.682809,0.743371,0.688400,0.833623,0.862330,0.705125,0.836494,0.879930,0.874274,0.732470
3,3,Отвратительное отношение к клиентам.,0.782251,0.777538,0.720639,0.725874,0.751448,0.762714,0.844817,0.807196,0.800240,0.835779,0.663675,0.629532,0.798442,0.841082,1.004477,0.865200,0.738199,1.090454,0.850974,0.892693,0.695038,0.807777,0.258026,0.000000,0.922865,0.779744,0.825767,0.782275,0.691542,0.914264,0.831290,0.880092,0.871679,0.789691,0.762702,0.740960,0.941819,0.923217,...,0.765360,0.854620,0.628396,0.631946,0.730359,0.977758,0.805062,0.830895,0.800755,0.844817,0.866692,0.807196,0.682487,0.751766,0.770594,0.607582,0.798056,0.825767,0.815918,0.696495,0.718478,0.765460,0.751448,0.903471,0.710939,0.798442,0.891907,0.818867,0.807326,0.884912,0.645304,0.700656,0.770503,0.865670,0.764885,0.76

In [188]:
#!g1.1
train_df = emb_sbert.copy()
train_df = pd.concat((
    train_df, 
    dist_sent_final.iloc[:, 2:], 
    # dist_cat_final.iloc[:, 2:]
), axis=1)

train_df['sentiment'] = train.sentiment
train_df['1category'] = train['1category']

d_sent = {
    '?': 0,
    '−' :1,
    '+': 2
}
d_cat = {'Communication':0, 'Quality':1, 'Price':2, 'Safety':3}
train_df = train_df[train_df['1category'] != '?']
train_df.sentiment = train_df.sentiment.map(lambda x: d_sent[x])
train_df['1category'] = train_df['1category'].map(lambda x: d_cat[x])


In [189]:
#!g1.1
768 + 80 + 1 + 138 - 2

985

In [190]:
#!g1.1
train_df = train_df.drop(columns=['1category'])
train_df

,sbert_0,sbert_1,sbert_2,sbert_3,sbert_4,sbert_5,sbert_6,sbert_7,sbert_8,sbert_9,sbert_10,sbert_11,sbert_12,sbert_13,sbert_14,sbert_15,sbert_16,sbert_17,sbert_18,sbert_19,sbert_20,sbert_21,sbert_22,sbert_23,sbert_24,sbert_25,sbert_26,sbert_27,sbert_28,sbert_29,sbert_30,sbert_31,sbert_32,sbert_33,sbert_34,sbert_35,sbert_36,sbert_37,sbert_38,sbert_39,...,dist_neg_5,dist_neg_6,dist_neg_7,dist_neg_8,dist_neg_9,dist_neg_10,dist_neg_11,dist_neg_12,dist_neg_13,dist_neg_14,dist_neg_15,dist_neg_16,dist_neg_17,dist_net_0,dist_net_1,dist_net_2,dist_net_3,dist_net_4,dist_net_5,dist_net_6,dist_net_7,dist_net_8,dist_net_9,dist_net_10,dist_net_11,dist_net_12,dist_net_13,dist_net_14,dist_net_15,dist_net_16,dist_net_17,dist_net_18,dist_net_19,dist_net_20,dist_net_21,dist_net_22,dist_net_23,dist_net_24,dist_net_25,sentiment
0,-0.260432,-0.713181,0.003374,-0.555725,-0.040731,-0.649877,0.681974,-0.492740,0.182050,-0.232997,-0.812267,0.105695,-0.658094,0.458973,0.376190,-0.546793,0.880897,-1.016082,-0.273517,0.086956,0.107551,-0.566165,-0.579776,0.868737,-0.350201,0.175540,0.197782,-0.576191,0.394082,-0.528489,0.534617,0.874266,-0.994191,1.475890,-1.130669,-0.193522,-0.112922,1.634456,-0.306568,-0.049407,...,0.826995,0.819750,0.736176,0.848231,0.660029,0.456691,0.759870,0.474292,0.780699,0.634758,0.423451,0.797976,0.680918,0.440418,0.794999,0.570097,0.745708,0.456691,0.573373,0.722134,0.675660,0.604929,0.681406,0.924446,0.698424,0.551746,0.794674,0.625063,0.660604,0.760979,0.739961,0.817767,0.814103,0.744761,0.541154,0.584369,0.726199,0.842304,0.952256,2
3,-0.187049,-1.172277,-0.252848,-0.020091,-1.145047,-0.293424,0.796849,-0.893079,0.184771,-0.879934,-0.604819,0.378867,-0.275533,-0.516072,0.163134,-0.513828,0.666621,-0.176098,-0.085796,-1.278306,0.593058,-0.164535,-0.908730,0.594817,0.423382,-0.573923,0.372415,0.206064,0.970025,0.666710,0.069733,-0.145418,-0.951699,0.404901,-0.695958,0.250444,0.353053,1.656136,0.011415,-0.654960,...,0.629532,0.847164,0.258026,0.751795,0.737491,0.720639,0.000000,0.683376,0.937594,0.841082,0.766761,0.825767,0.782251,0.705851,0.639173,0.763197,0.853429,0.720639,0.794713,0.745125,0.765460,0.831290,0.807196,0.973786,0.891967,0.792868,0.908751,0.833514,0.798442,0.865200,0.834056,0.818867,0.727231,0.871679,0.828155,0.659338,0.830752,0.907560,0.967083,1
4,-0.622859,-0.543196,0.239860,-0.251940,-0.344115,-0.174159,0.728634,-0.675047,1.129310,-0.788626,-0.637421,0.142306,1.535083,-0.247185,-0.406887,-0.307516,1.470772,-1.677171,-0.291206,0.546568,0.218565,0.057009,-0.869788,0.227676,0.371716,-0.280134,0.511731,-1.062239,0.620145,-0.455612,0.664995,0.932008,-0.348920,1.383520,-0.290409,-0.113911,0.164629,0.670170,-0.247544,-0.413349,...,0.771850,0.892109,0.748005,0.684793,0.675092,0.355247,0.878478,0.522335,0.587327,0.566074,0.563046,0.715301,0.636743,0.520143,0.748258,0.688514,0.836714,0.355247,0.549932,0.784346,0.783925,0.933354,0.703201,0.823529,0.655420,0.656092,0.512498,0.515005,0.614425,0.861087,0.797814,0.830979,0.681806,0.828330,0.587285,0.588129,0.626695,0.801969,0.978324,2
5,-0.400690,-0.257951,-0.052696,0.149262,0.605822,-0.570054,0.768724,-1.126514,0.429213,-0.160242,-0.199646,0.266961,0.512638,-0.332451,-0.008806,-0.564237,0.868642,-1.397380,-1.054094,0.407054,-0.219646,-0.662999,-1.068739,0.873505,-0.290399,-0.803336,0.889715,-0.721595,0.466987,-0.285529,1.083076,0.571411,-0.762143,1.594564,-0.856970,-0.209349,0.730737,0.672109,-0.410803,-0.830306,...,0.799270,0.881793,0.774490,0.900568,0.675092,0.534646,0.832502,0.659200,0.762063,0.766739,0.609747,0.831547,0.808925,0.524886,0.914083,0.893751,0.867858,0.534646,0.713805,0.784346,0.783925,0.901432,0.806789,0.688554,0.781959,0.656092,0.648395,0.515005,0.872797,0.948678,0.857385,0.915817,0.830674,0.952837,0.599496,0.502110,0.783856,0.822189,1.019553,2
6,-0.389520,-1.367927,0.007435,-0.618673,-1.815774,-0.696406,0.743402,0.005171,0.286884,-0.281013,-0.234664,0.639049,-0.392119,0.004720,0.225212,-0.727631,0.864375,0.715222,-0.150142,0.105714,1.021221,-0.530921,-0.0

In [183]:
#!g1.4
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import roc_auc_score

automl = TabularAutoML(
    timeout=3600,
    gpu_ids=0,
    memory_limit=80,
    cpu_limit=32,
    task = Task(
        name = 'multiclass',
        metric='auc'
))

oof_pred = automl.fit_predict(
    train_data=train_df,
    roles = {'target': 'sentiment'},
    verbose=1
)

Execute error: Failed to allocate servant g1.4: internal error. Please try again

In [ ]:
#!g1.1
pd.set_option('display.max_rows', 100)
automl.get_feature_scores()[:100]

In [ ]:
#!g1.1
import joblib

# joblib.dump(automl, '/home/jupyter/mnt/s3/bank-data/models/cat1_sbert_dist_cat_final_8672.pkl')

In [221]:
#!g1.1
automl = joblib.load('/home/jupyter/mnt/s3/bank-data/models/cat1_sbert_dist_cat_final_8672.pkl')

In [222]:
#!g1.1
test_emb_sbert = pd.read_csv('/home/jupyter/mnt/s3/bank-data/features/TEST_ONE_embeddings_sbert.csv')
test_dist_cat_final = pd.read_csv('/home/jupyter/mnt/s3/bank-data/features/TEST_distances_category.csv')
test_dist_sent_final = pd.read_csv('/home/jupyter/mnt/s3/bank-data/features/TEST_distances_sentiment.csv')

In [223]:
#!g1.1
print(test_emb_sbert.values.shape)
print(test_dist_cat_final.values.shape)
print(test_dist_sent_final.values.shape)

(949, 768)
(949, 138)
(949, 82)


In [224]:
#!g1.1
#!g1.1
test_df = pd.concat((
    test_emb_sbert, 
    test_dist_sent_final.iloc[:, 2:], 
    # test_dist_cat_final.iloc[:, 2:]
), axis=1)
test_df

,sbert_0,sbert_1,sbert_2,sbert_3,sbert_4,sbert_5,sbert_6,sbert_7,sbert_8,sbert_9,sbert_10,sbert_11,sbert_12,sbert_13,sbert_14,sbert_15,sbert_16,sbert_17,sbert_18,sbert_19,sbert_20,sbert_21,sbert_22,sbert_23,sbert_24,sbert_25,sbert_26,sbert_27,sbert_28,sbert_29,sbert_30,sbert_31,sbert_32,sbert_33,sbert_34,sbert_35,sbert_36,sbert_37,sbert_38,sbert_39,...,dist_neg_4,dist_neg_5,dist_neg_6,dist_neg_7,dist_neg_8,dist_neg_9,dist_neg_10,dist_neg_11,dist_neg_12,dist_neg_13,dist_neg_14,dist_neg_15,dist_neg_16,dist_neg_17,dist_net_0,dist_net_1,dist_net_2,dist_net_3,dist_net_4,dist_net_5,dist_net_6,dist_net_7,dist_net_8,dist_net_9,dist_net_10,dist_net_11,dist_net_12,dist_net_13,dist_net_14,dist_net_15,dist_net_16,dist_net_17,dist_net_18,dist_net_19,dist_net_20,dist_net_21,dist_net_22,dist_net_23,dist_net_24,dist_net_25
0,-0.358358,-1.125138,1.061932,-0.312551,-1.059808,-0.381704,0.205776,0.950917,0.332628,0.176699,0.852082,0.063350,-0.401275,0.387599,-0.123161,-0.526430,1.021837,0.704661,0.621837,0.184350,0.837256,0.053336,0.027138,1.676097,0.831347,1.105354,1.341350,0.072375,0.399156,0.253993,-1.168963,1.180240,-0.879872,1.352175,-0.301183,-0.230800,-1.000994,0.897466,0.043420,-0.360395,...,0.788189,0.000000,0.926941,0.660773,0.579087,0.735379,0.740286,0.629532,0.624775,0.631141,0.666591,0.690462,0.743183,0.677578,0.599909,0.758836,0.565266,0.795067,0.740286,0.677876,0.795155,0.789297,0.605609,0.766822,0.884960,0.719475,0.734966,0.793013,0.800451,0.679445,0.893881,0.700463,0.637153,0.699830,0.492132,0.665720,0.712300,0.738902,0.780750,0.766723
1,-0.515058,-1.429527,0.210361,-0.678841,-1.446823,-0.367436,0.692850,0.055999,0.435522,-0.795941,-0.104977,0.291527,-0.272086,0.093439,0.152968,-0.268569,0.877290,0.572937,-0.491085,0.911687,1.496500,-0.202018,0.462402,0.329956,0.234865,0.090729,1.114865,-0.603594,0.729826,0.187237,-0.328089,0.384389,0.172592,0.821125,-0.517098,0.151736,-0.884064,1.069745,-0.166240,0.000984,...,0.776628,0.772521,0.751454,0.650807,0.797097,0.704386,0.595082,0.652252,0.658175,0.689887,0.738748,0.618350,0.841730,0.783601,0.618035,0.551995,0.622963,0.684169,0.595082,0.791053,0.731735,0.540176,0.597585,0.734371,0.781974,0.742424,0.734966,0.553876,0.708226,0.767444,0.686139,0.700463,0.715056,0.617223,0.647917,0.705318,0.554111,0.803766,0.691168,0.766723
2,0.328066,-0.494250,-0.280689,-0.568823,0.257320,-0.883106,0.634842,-0.844393,0.403455,-0.614361,-0.606540,0.364345,0.619336,0.613637,-0.248892,-0.100996,1.525906,-1.799458,-0.450333,0.231853,-0.147398,-0.501690,-0.511389,0.917927,0.355324,-0.644957,0.141850,-1.023596,0.511554,-0.628307,1.438898,0.251153,-0.804105,1.271368,-0.722150,-0.187182,1.485749,1.247402,-0.728453,-0.642039,...,0.786399,0.734619,0.940780,0.692634,0.899791,0.798961,0.355247,0.805325,0.798989,0.941881,0.779798,0.698203,0.873932,0.842992,0.607952,0.833235,0.972665,0.836714,0.355247,0.690695,0.799099,0.807711,0.945400,0.772513,1.016993,0.793984,0.739183,0.512498,0.691172,0.894548,0.967482,0.903604,0.830979,0.854532,0.828330,0.680935,0.549706,0.848457,0.801969,0.978324
3,-0.441643,-1.160997,0.705099,-0.756106,-0.723589,-0.440724,0.305229,0.192878,-0.539822,0.058577,1.072889,0.349201,-0.342788,0.306226,-0.155886,-0.052916,0.540643,0.649029,-0.236799,-0.001747,1.485992,-0.495886,0.567910,1.186099,0.543641,0.291967,1.828711,0.325912,0.581785,0.919790,-0.143728,0.720854,-0.617798,1.194606,-1.064977,-0.307401,-0.853453,1.196380,0.220362,-0.526053,...,0.694668,0.725285,0.940780,0.678083,0.811209,0.726381,0.000000,0.743101,0.681834,0.814030,0.762824,0.629111,0.694865,0.779499,0.511598,0.679135,0.622963,0.697742,0.000000,0.690695,0.768014,0.661509,0.771110,0.630169,0.771993,0.699516,0.649229,0.512498,0.680034,0.640833,0.882429,0.655976,0.764873,0.699830,0.771844,0.615651,0.473157,0.616583,0.776327,0.905895
4,-0.788094,-0.999858,0.386479,-0.529097,-0.570581,-0.405728,0.491191,0.435572,0.348067,-0.187007,-0.066532,-0.001418,-0.755029,-0.481563,-0.033514,-0.491205,0.501949,0.317717,0.648954,-0.235072,-0.128626,-0

In [226]:
#!g1.1
predict = automl.predict(test_df)
df = pd.DataFrame(predict.data, columns=['?', '−', '+'])
df

,?,−,+
0,0.566018,0.030448,0.403534
1,0.881945,0.079325,0.038730
2,0.228228,0.567896,0.203875
3,0.926895,0.020850,0.052256
4,0.121744,0.789561,0.088695
...,...,...,...
944,0.574394,0.271594,0.154012
945,0.208906,0.160423,0.630671
946,0.204237,0.319247,0.476516
947,0.992975,0.003544,0.003481


In [227]:
#!g1.1


In [220]:
#!g1.1
df.to_csv('/home/jupyter/mnt/s3/bank-data/test_results/sentiment.csv')

In [ ]:
#!g1.1
